In [1]:
import numpy as np
import pandas as pd
import cv2
import pytesseract

import os
from glob import glob
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')


In [2]:
imgPaths = glob('./dataset/*.jpg')

In [3]:
imgPaths[0]

'./dataset\\print_1.jpg'

In [4]:
all_receipt = pd.DataFrame(columns=['id','text'])

for imgPath in  tqdm(imgPaths,desc='BusinessCard'):
    
    #imgPath = imgPaths[0]
    _, filename = os.path.split(imgPath)
    # extract data and text 
    image = cv2.imread(imgPath)
    data = pytesseract.image_to_data(image, lang='tha+eng', config='--psm 3')
    dataList = list(map(lambda x: x.split('\t'),data.split('\n')))
    df = pd.DataFrame(dataList[1:],columns=dataList[0])
    df.dropna(inplace=True)
    df['conf'] = df['conf'].astype(float)

    useFulData = df.query('conf >= 30')

    # Dataframe
    receipt = pd.DataFrame()
    receipt['text'] = useFulData['text']
    receipt['id'] = filename
    
    # concatenation
    all_receipt = pd.concat((all_receipt,receipt))

BusinessCard:   0%|          | 0/265 [00:00<?, ?it/s]

BusinessCard: 100%|██████████| 265/265 [11:32<00:00,  2.61s/it]


In [5]:
all_receipt.to_csv('Receipt.csv',index=False)